In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from transformers import BertTokenizer, BertModel


In [3]:
from sklearn.model_selection import train_test_split

import pandas as pd


In [4]:
df = pd.read_table("../tamil_offensive_train.tsv")

train_df, test_df = train_test_split(df,test_size=0.2,random_state=32)

In [5]:
## Will be useful later
## Based on https://www.kaggle.com/andradaolteanu/ii-commonlit-bert-vs-roberta-w-b-testing#3.-The-Dataset
class TamilOffensiveDataset(Dataset):
    def __init__(self,texts,targets,tokenizer,max_len,is_test):
        self.text= texts
        self.target = targets
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.is_test = is_test 
        
    def __len__(self):
        return len(self.text)
    
    
    def __getitem__(self,i):
        '''This helps in loading data
            i:indexes to be loaded'''
        
        batch_text = str(self.text[i])
        inputs = self.tokenizer(batch_text,
                                max_length = self.max_len,
                                padding = "max_length",
                                truncation = True)
        
        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]
            
        if self.is_test == True:
            return {"input_ids" : torch.tensor(ids, dtype=torch.long),
                    "attention_mask" : torch.tensor(mask, dtype=torch.long)}
        else:
            target = self.target[i]
            return {"input_ids" : torch.tensor(ids, dtype=torch.long),
                    "attention_mask" : torch.tensor(mask, dtype=torch.long),
                    "target" : torch.tensor(target, dtype=torch.float)}

In [6]:
train_texts = train_df['text'].values.tolist()
train_targets = train_df['category'].values.tolist()
train_targets = [ 1  if t == 'OFF' else 0 for t in train_targets]

train_df['num_words'] = train_df['text'].str.split().apply(len)

train_df.num_words.quantile(0.98)

/home/deepak/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


52.0

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')



In [8]:
features =[]

In [9]:
model = BertModel.from_pretrained("bert-base-multilingual-uncased")

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
sample_text = train_texts[:64]
#need to run this in batches to extract feature for entire dataset
t = tokenizer(sample_text,max_length = 128,
                                padding = "max_length",
                                truncation = True)

In [11]:
input_ids = torch.tensor(t['input_ids'])
attention_masks = torch.tensor(t['attention_mask'])

In [12]:
with torch.no_grad():
    lhs,pooler_output = model(input_ids,attention_masks,return_dict=False)

In [14]:
#feature for CLS token - this can be used as a feature
pooler_output.shape

torch.Size([64, 768])

In [16]:
#feature for entire sequence, we can take average of these and use as feature
lhs.shape

torch.Size([64, 128, 768])

In [17]:
#Use one of the above as features and build a logistic regression classifier